## Imports

In [43]:
import copy
import itertools
import logging
import math
from functools import partial
from pathlib import Path
from typing import Dict
import json

import hydra
import matplotlib
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import omegaconf
import pytorch_lightning
import seaborn as sns
import torch  # noqa
import wandb
from hydra.utils import instantiate
from matplotlib import tri
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from omegaconf import DictConfig
from pytorch_lightning import LightningModule
from scipy.stats import qmc
from torch.utils.data import DataLoader, Subset, SubsetRandomSampler
from tqdm import tqdm

from nn_core.callbacks import NNTemplateCore
from nn_core.common import PROJECT_ROOT
from nn_core.common.utils import seed_index_everything
from nn_core.model_logging import NNLogger

import ccmm  # noqa
from ccmm.matching.utils import (
    apply_permutation_to_statedict,
    get_all_symbols_combinations,
    load_permutations,
    perm_indices_to_perm_matrix,
    plot_permutation_history_animation,
    restore_original_weights,
)
from ccmm.utils.utils import (
    fuse_batch_norm_into_conv,
    get_interpolated_loss_acc_curves,
    l2_norm_models,
    linear_interpolate,
    load_model_from_info,
    map_model_seed_to_symbol,
    normalize_unit_norm,
    project_onto,
    save_factored_permutations,
    vector_to_state_dict,
)

In [44]:
plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
    }
)
sns.set_context("talk")

cmap_name = "coolwarm_r"

from ccmm.utils.plot import Palette

palette = Palette(f"{PROJECT_ROOT}/misc/palette2.json")
palette

{'light red': '#c05761',
 'green': '#2a9d8f',
 'yellow': '#e9c46a',
 'dark blue': '#264653',
 'red': '#941c2f',
 'orange': '#e76f51',
 'violet': '#734f5a',
 'light orange': '#f4a261'}

In [45]:
logging.getLogger("lightning.pytorch").setLevel(logging.WARNING)
logging.getLogger("torch").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning.accelerators.cuda").setLevel(logging.WARNING)
pylogger = logging.getLogger(__name__)

## Configuration

In [46]:
%load_ext autoreload
%autoreload 2

import hydra
from hydra import initialize, compose
from typing import Dict, List

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path=str("../conf"), job_name="matching")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


hydra.initialize()

In [47]:
cfg = compose(config_name="matching", overrides=["model=mlp", "dataset=emnist"])

In [48]:
core_cfg = cfg  # NOQA
cfg = cfg.matching

seed_index_everything(cfg)

2025-01-03 08:28:48 INFO     Global seed set to 1273642419                      ]8;id=418697;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/utilities/seed.py\pytorch_lightning.utilities.seed]8;;\:]8;id=267232;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/utilities/seed.py#71\71]8;;\

                    INFO     Setting seed 1273642419 from seeds[1]                         ]8;id=874177;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/nn_core/common/utils.py\nn_core.common.utils]8;;\:]8;id=339757;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/nn_core/common/utils.py#107\107]8;;\

1273642419

## Hyperparameters

In [49]:
num_test_samples = 5000
num_train_samples = 10000

## Load dataset

In [50]:
transform = instantiate(core_cfg.dataset.test.transform)

train_dataset = instantiate(core_cfg.dataset.train, transform=transform)
test_dataset = instantiate(core_cfg.dataset.test, transform=transform)

train_subset = Subset(train_dataset, list(range(num_train_samples)))
train_loader = DataLoader(train_subset, batch_size=5000, num_workers=cfg.num_workers)

test_subset = Subset(test_dataset, list(range(num_test_samples)))

test_loader = DataLoader(test_subset, batch_size=1000, num_workers=cfg.num_workers)

/media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/data/EMNIST/raw/emnist-letters-train-images-idx3-ubyte /media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/data/EMNIST/raw/emnist-letters-train-labels-idx1-ubyte
/media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/data/EMNIST/raw/emnist-letters-test-images-idx3-ubyte /media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/data/EMNIST/raw/emnist-letters-test-labels-idx1-ubyte


In [51]:
trainer = instantiate(cfg.trainer, enable_progress_bar=False, enable_model_summary=False)

/home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(


                    INFO     GPU available: True (cuda), used: True      ]8;id=174676;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=790783;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1751\1751]8;;\

                    INFO     TPU available: False, using: 0 TPU cores    ]8;id=447211;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=530377;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1754\1754]8;;\

                    INFO     IPU available: False, using: 0 IPUs         ]8;id=143249;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=263840;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1757\1757]8;;\

                    INFO     HPU available: False, using: 0 HPUs         ]8;id=768303;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=365854;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1760\1760]8;;\

## Load models

In [52]:
from ccmm.utils.utils import load_model_from_artifact

run = wandb.init(project=core_cfg.core.project_name, entity=core_cfg.core.entity, job_type="matching")

# {a: 1, b: 2, c: 3, ..}
symbols_to_seed: Dict[int, str] = {map_model_seed_to_symbol(seed): seed for seed in cfg.model_seeds}

artifact_path = (
    lambda seed: f"{core_cfg.core.entity}/{core_cfg.core.project_name}/{core_cfg.dataset.name}_{core_cfg.model.model_identifier}_{seed}:latest"
)

# {a: model_a, b: model_b, c: model_c, ..}
models: Dict[str, LightningModule] = {
    map_model_seed_to_symbol(seed): load_model_from_artifact(run, artifact_path(seed)) for seed in cfg.model_seeds
}
model_orig_weights = {symbol: copy.deepcopy(model.model.state_dict()) for symbol, model in models.items()}

num_models = len(models)

pylogger.info(f"Using {num_models} models with architecture {core_cfg.model.model_identifier}")

merging_time,▁
merging_time,2.28722


2025-01-03 08:28:55 INFO     Trying to load                                                    ]8;id=203790;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/utils/utils.py\ccmm.utils.utils]8;;\:]8;id=464800;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/utils/utils.py#264\264]8;;\
                             gladia/cycle-consistent-model-merging/EMNIST_MLP_1:latest                             

wandb:   1 of 1 files downloaded.  


2025-01-03 08:28:56 INFO     Trying to load                                                    ]8;id=44469;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/utils/utils.py\ccmm.utils.utils]8;;\:]8;id=758767;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/utils/utils.py#264\264]8;;\
                             gladia/cycle-consistent-model-merging/EMNIST_MLP_2:latest                             

wandb:   1 of 1 files downloaded.  


2025-01-03 08:28:58 INFO     Using 2 models with architecture MLP                                       ]8;id=140771;file:///tmp/ipykernel_11089/4100191972.py\__main__]8;;\:]8;id=576061;file:///tmp/ipykernel_11089/4100191972.py#20\20]8;;\

## Match models

In [53]:
# always permute the model having larger character order, i.e. c -> b, b -> a and so on ...
from ccmm.matching.matcher import GitRebasinMatcher
from ccmm.matching.utils import get_inverse_permutations

symbols = set(symbols_to_seed.keys())
sorted_symbols = sorted(symbols, reverse=False)
fixed_symbol, permutee_symbol = "a", "b"
fixed_model, permutee_model = models[fixed_symbol].cpu(), models[permutee_symbol].cpu()

In [54]:
from ccmm.matching.permutation_spec import MLPPermutationSpecBuilder, CNNPermutationSpecBuilder

permutation_spec_builder = MLPPermutationSpecBuilder(num_hidden_layers=4)
# permutation_spec_builder = CNNPermutationSpecBuilder()
permutation_spec = permutation_spec_builder.create_permutation_spec()

ref_model = list(models.values())[0]
assert set(permutation_spec.layer_and_axes_to_perm.keys()) == set(ref_model.model.state_dict().keys())

### Naive

In [55]:
restore_original_weights(models, model_orig_weights)

In [56]:
from ccmm.matching.matcher import DummyMatcher

matcher = DummyMatcher(name="naive", permutation_spec=permutation_spec)

In [57]:
# dicts for permutations and permuted params, D[a][b] refers to the permutation/params to map b -> a
naive_permutations = {symb: {other_symb: None for other_symb in symbols.difference(symb)} for symb in symbols}

naive_permutations[fixed_symbol][permutee_symbol], perm_history = matcher(
    fixed=fixed_model.model, permutee=permutee_model.model
)

naive_permutations[permutee_symbol][fixed_symbol] = get_inverse_permutations(
    naive_permutations[fixed_symbol][permutee_symbol]
)

In [58]:
from scripts.evaluate_matched_models import evaluate_pair_of_models

restore_original_weights(models, model_orig_weights)

updated_params = {fixed_symbol: {permutee_symbol: None}}

pylogger.info(f"Permuting model {permutee_symbol} into {fixed_symbol}.")

# perms[a, b] maps b -> a
updated_params[fixed_symbol][permutee_symbol] = apply_permutation_to_statedict(
    permutation_spec, naive_permutations[fixed_symbol][permutee_symbol], models[permutee_symbol].model.state_dict()
)
restore_original_weights(models, model_orig_weights)

lambdas = [0.0, 0.5, 1]  # np.linspace(0, 1, num=4)

                    INFO     Permuting model b into a.                                                   ]8;id=131413;file:///tmp/ipykernel_11089/3939703209.py\__main__]8;;\:]8;id=654578;file:///tmp/ipykernel_11089/3939703209.py#7\7]8;;\

In [59]:
naive_results = evaluate_pair_of_models(
    models,
    fixed_symbol,
    permutee_symbol,
    updated_params,
    train_loader,
    test_loader,
    lambdas,
    core_cfg,
)

/home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(


                    INFO     GPU available: True (cuda), used: True      ]8;id=77612;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=645076;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1751\1751]8;;\

                    INFO     TPU available: False, using: 0 TPU cores    ]8;id=197043;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=679197;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1754\1754]8;;\

                    INFO     IPU available: False, using: 0 IPUs         ]8;id=426806;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=325846;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1757\1757]8;;\

                    INFO     HPU available: False, using: 0 HPUs         ]8;id=17192;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=489511;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1760\1760]8;;\

...

 67%|██████▋   | 2/3 [00:03<00:01,  1.81s/it]

2025-01-03 08:29:02 INFO     Evaluating interpolated model with lambda: 1                      ]8;id=532951;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/utils/utils.py\ccmm.utils.utils]8;;\:]8;id=653817;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/utils/utils.py#168\168]8;;\

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         acc/test          │    0.9769999980926514     │
│         loss/test         │    0.05489429831504822    │
└───────────────────────────┴───────────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         acc/test          │    0.9241999983787537     │
│         loss/test         │    0.49278753995895386    │
└───────────────────────────┴───────────────────────────┘

100%|██████████| 3/3 [00:05<00:00,  1.80s/it]


### Git Re-Basin

In [60]:
restore_original_weights(models, model_orig_weights)

In [61]:
# dicts for permutations and permuted params, D[a][b] refers to the permutation/params to map b -> a
gitrebasin_permutations = {symb: {other_symb: None for other_symb in symbols.difference(symb)} for symb in symbols}

matcher = GitRebasinMatcher(name="git_rebasin", permutation_spec=permutation_spec)
gitrebasin_permutations[fixed_symbol][permutee_symbol], perm_history = matcher(
    fixed=fixed_model.model.cpu(), permutee=permutee_model.model.cpu()
)

gitrebasin_permutations[permutee_symbol][fixed_symbol] = get_inverse_permutations(
    gitrebasin_permutations[fixed_symbol][permutee_symbol]
)

Weight matching:   0%|          | 0/100 [00:00<?, ?it/s]

2025-01-03 08:29:04 INFO     Iteration 0, Permutation P_0: 921.8171997070312      ]8;id=210054;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/matching/weight_matching.py\ccmm.matching.weight_matching]8;;\:]8;id=849849;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/matching/weight_matching.py#176\176]8;;\

                    INFO     Iteration 0, Permutation P_1: 345.47674560546875     ]8;id=617293;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/matching/weight_matching.py\ccmm.matching.weight_matching]8;;\:]8;id=537542;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/matching/weight_matching.py#176\176]8;;\

                    INFO     Iteration 0, Permutation P_2: 167.64512634277344     ]8;id=5815;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/matching/weight_matching.py\ccmm.matching.weight_matching]8;;\:]8;id=881666;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/matching/weight_matching.py#176\176]8;;\

                    INFO     Iteration 0, Permutation P_3: 66.29916381835938      ]8;id=625239;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/matching/weight_matching.py\ccmm.matching.weight_matching]8;;\:]8;id=909407;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/matching/weight_matching.py#176\176]8;;\

...

                    INFO     Iteration 21, Permutation P_0: 0.0                   ]8;id=498666;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/matching/weight_matching.py\ccmm.matching.weight_matching]8;;\:]8;id=252915;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/matching/weight_matching.py#176\176]8;;\

                    INFO     Iteration 21, Permutation P_2: 0.0                   ]8;id=76738;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/matching/weight_matching.py\ccmm.matching.weight_matching]8;;\:]8;id=363439;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/matching/weight_matching.py#176\176]8;;\

                    INFO     Iteration 21, Permutation P_1: 0.0                   ]8;id=63351;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/matching/weight_matching.py\ccmm.matching.weight_matching]8;;\:]8;id=150495;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/matching/weight_matching.py#176\176]8;;\

Weight matching:  21%|██        | 21/100 [00:02<00:08,  9.66it/s]

Function __call__ Took 2.1750 seconds


In [62]:
from scripts.evaluate_matched_models import evaluate_pair_of_models

restore_original_weights(models, model_orig_weights)

updated_params = {fixed_symbol: {permutee_symbol: None}}

pylogger.info(f"Permuting model {permutee_symbol} into {fixed_symbol}.")

# perms[a, b] maps b -> a
updated_params[fixed_symbol][permutee_symbol] = apply_permutation_to_statedict(
    permutation_spec, gitrebasin_permutations[fixed_symbol][permutee_symbol], models[permutee_symbol].model.state_dict()
)
restore_original_weights(models, model_orig_weights)

lambdas = [0.0, 0.5, 1.0]

gitrebasin_results = evaluate_pair_of_models(
    models,
    fixed_symbol,
    permutee_symbol,
    updated_params,
    train_loader,
    test_loader,
    lambdas,
    core_cfg,
)

                    INFO     Permuting model b into a.                                                   ]8;id=137991;file:///tmp/ipykernel_11089/290635983.py\__main__]8;;\:]8;id=842570;file:///tmp/ipykernel_11089/290635983.py#7\7]8;;\

/home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(


                    INFO     GPU available: True (cuda), used: True      ]8;id=758468;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=539365;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1751\1751]8;;\

                    INFO     TPU available: False, using: 0 TPU cores    ]8;id=363952;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=556335;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1754\1754]8;;\

                    INFO     IPU available: False, using: 0 IPUs         ]8;id=820550;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=33189;file:///home/donato/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1757\1757]8;;\

...

 67%|██████▋   | 2/3 [00:03<00:01,  1.91s/it]

2025-01-03 08:29:10 INFO     Evaluating interpolated model with lambda: 1.0                    ]8;id=279204;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/utils/utils.py\ccmm.utils.utils]8;;\:]8;id=119238;file:///media/donato/Extra-storage/Code/model-merging/cycle-consistent-model-merging/src/ccmm/utils/utils.py#168\168]8;;\

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         acc/test          │    0.9769999980926514     │
│         loss/test         │    0.05489429831504822    │
└───────────────────────────┴───────────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         acc/test          │    0.9241999983787537     │
│         loss/test         │    0.49278751015663147    │
└───────────────────────────┴───────────────────────────┘

100%|██████████| 3/3 [00:05<00:00,  1.92s/it]


## QAP

In [63]:
restore_original_weights(models, model_orig_weights)

## Quadratic

In [76]:
from enum import auto
from backports.strenum import StrEnum


class EigenvectorPostprocess(StrEnum):
    LEORDEANU = auto()
    LAP = auto()


class DiagContent(StrEnum):
    """Enum for diagonal content of affinity matrix"""

    ONES = auto()
    SIMILARITIES = auto()


def build_affinity_matrix_vectorized(Wa, Wb, diag: DiagContent):
    Wa = torch.tensor(Wa)
    Wb = torch.tensor(Wb)

    num_neurons = Wa.size(0)
    num_matchings = num_neurons**2

    # Compute all pairwise Euclidean distances for Wa and Wb
    Wa_distances = torch.cdist(Wa, Wa, p=2)
    Wb_distances = torch.cdist(Wb, Wb, p=2)

    # Prepare the distance matrices for broadcasting
    Wa_distances = Wa_distances.view(num_neurons, 1, num_neurons, 1).expand(-1, num_neurons, -1, num_neurons)
    Wb_distances = Wb_distances.view(1, num_neurons, 1, num_neurons).expand(num_neurons, -1, num_neurons, -1)

    S = torch.exp(-torch.abs(Wa_distances - Wb_distances) / 1e-2)

    S = S.reshape(num_matchings, num_matchings)

    if diag == DiagContent.ONES:
        diag_matrix = torch.eye(num_matchings)

    elif diag == DiagContent.SIMILARITIES:
        Wa_Wb_distances = torch.cdist(Wa, Wb, p=2)
        Wa_Wb_sim = torch.exp(-torch.abs(Wa_Wb_distances) / 1e-2)

        Wa_Wb_sim = Wa_Wb_sim.reshape(num_matchings, 1)

        diag_matrix = torch.diag(Wa_Wb_sim.squeeze())

    mask = torch.eye(num_matchings, dtype=torch.bool)
    S[mask] = 0

    S = S + diag_matrix

    return S.numpy()

In [77]:
from torch import Tensor


def extract_matching_leordeanu(principal_eigenvector: Tensor):
    """
    principal_eigenvector: shape (num_neurons, num_neurons)
    """

    num_neurons = principal_eigenvector.shape[0]

    # Initialize the solution vector
    x = torch.zeros((num_neurons, num_neurons)).type_as(principal_eigenvector).long()

    # Initialize masks for rows and columns
    row_mask = torch.ones(num_neurons, dtype=torch.bool)
    col_mask = torch.ones(num_neurons, dtype=torch.bool)

    while True:
        # Apply masks to principal eigenvector
        masked_principal_eigenvector = principal_eigenvector.clone()
        masked_principal_eigenvector[~row_mask, :] = 0
        masked_principal_eigenvector[:, ~col_mask] = 0

        # Find the maximum value and its index
        flat_index = masked_principal_eigenvector.argmax()

        i, j = np.unravel_index(flat_index.item(), (num_neurons, num_neurons))

        assignment_value = masked_principal_eigenvector[(i, j)]
        if assignment_value == 0:
            break

        # Update the solution vector
        x[i, j] = 1

        # Update the masks to exclude row i and column j
        row_mask[i] = False
        col_mask[j] = False

    return x

In [78]:
from scipy.optimize import linear_sum_assignment
import scipy


def extract_matching_lap(principal_eigenvector):
    num_neurons = principal_eigenvector.shape[0]

    principal_eigenvector = principal_eigenvector.cpu().numpy()

    row_ind, col_ind = linear_sum_assignment(principal_eigenvector.max() - principal_eigenvector)
    P_AB = scipy.sparse.coo_matrix(
        (np.ones(num_neurons), (row_ind, col_ind)), shape=(num_neurons, num_neurons)
    ).toarray()

    return torch.tensor(P_AB)

In [79]:
from typing import Tuple
from ccmm.matching.func_maps import compute_eigenvectors, fit_func_map
from ccmm.matching.permutation_spec import PermutationSpec
from ccmm.matching.quadratic_matching import get_principal_eigenvector
from ccmm.matching.utils import get_permuted_param, perm_cols, perm_matrix_to_perm_indices, perm_rows
from ccmm.matching.weight_matching import (
    LayerIterationOrder,
    compute_weights_similarity,
    get_layer_iteration_order,
    solve_linear_assignment_problem,
)


def quadratic_matching(
    ps: PermutationSpec,
    fixed,
    permutee,
    max_iter=100,
    init_perm=None,
    layer_iteration_order: LayerIterationOrder = LayerIterationOrder.RANDOM,
    verbose=False,
    method="quadratic",
    eigenvec_postprocess: EigenvectorPostprocess = EigenvectorPostprocess.LEORDEANU,
):
    """
    Find a permutation of params_b to make them match params_a.

    :param ps: PermutationSpec
    :param target: the parameters to match
    :param to_permute: the parameters to permute
    """

    if not verbose:
        pylogger.setLevel(logging.WARNING)

    params_a, params_b = fixed, permutee

    perm_sizes = {}

    for p, params_and_axes in ps.perm_to_layers_and_axes.items():

        # p is the permutation matrix name, e.g. P_0, P_1, ..
        # params_and_axes is a list of tuples, each tuple contains the name of the parameter and the axis on which the permutation matrix acts

        # it is enough to select a single parameter and axis, since all the parameters permuted by the same matrix have the same shape
        ref_tuple = params_and_axes[0]
        ref_param_name = ref_tuple[0]
        ref_axis = ref_tuple[1]

        perm_sizes[p] = params_a[ref_param_name].shape[ref_axis]

    # initialize with identity permutation if none given
    all_perm_indices = {p: torch.arange(n) for p, n in perm_sizes.items()} if init_perm is None else init_perm
    # e.g. P0, P1, ..
    perm_names = list(all_perm_indices.keys())

    num_layers = len(perm_names)

    loss = {p: 1e3 for p in perm_names}

    for iteration in tqdm(range(max_iter), desc="Weight matching"):
        progress = False

        perm_order = get_layer_iteration_order(layer_iteration_order, num_layers)

        for p_ix in perm_order:
            print(f"Permuting {perm_names[p_ix]}")

            p = perm_names[p_ix]
            num_neurons = perm_sizes[p]

            # all the params that are permuted by this permutation matrix, together with the axis on which it acts
            # e.g. ('layer_0.weight', 0), ('layer_0.bias', 0), ('layer_1.weight', 0)..
            params_and_axes: List[Tuple[str, int]] = ps.perm_to_layers_and_axes[p]
            # sort by axis, so that we can permute the columns of the weight matrices first
            params_and_axes = sorted(params_and_axes, key=lambda x: x[1])
            # filter out bias
            params_and_axes = [x for x in params_and_axes if "bias" not in x[0]]

            # TODO: check if this is true in more complex architectures (probably not)
            assert len(params_and_axes) == 2, f"Expected 2 params, got {len(params_and_axes)}: {params_and_axes}"

            # AXES 0, CURRENT LAYER
            curr_params_name = params_and_axes[0][0]
            assert params_and_axes[0][1] == 0

            # (num_neurons, neuron_dim)
            w_a = copy.deepcopy(params_a[curr_params_name])
            w_b = copy.deepcopy(params_b[curr_params_name])
            assert w_a.shape == w_b.shape

            perms_to_apply = ps.layer_and_axes_to_perm[curr_params_name]

            col_perm_to_apply = perms_to_apply[1]

            if col_perm_to_apply is not None:
                perm_matrix = perm_indices_to_perm_matrix(all_perm_indices[col_perm_to_apply])
                print(
                    f"Permuting the columns of {curr_params_name} with {col_perm_to_apply}, shape: {perm_matrix.shape}"
                )

                w_b = perm_cols(w_b, perm_matrix.T)

            # AXES 1, NEXT LAYER
            next_params_name = params_and_axes[1][0]
            w_a_next = copy.deepcopy(params_a[next_params_name])
            w_b_next = copy.deepcopy(params_b[next_params_name])

            assert w_a_next.shape == w_b_next.shape

            perms_to_apply = ps.layer_and_axes_to_perm[next_params_name]
            perm_row_next_layer = perms_to_apply[0]

            if perm_row_next_layer is not None:
                w_b_next = perm_rows(w_b_next, perm_indices_to_perm_matrix(all_perm_indices[perm_row_next_layer]))

            w_a_next = torch.moveaxis(w_a_next, 1, 0).reshape((num_neurons, -1))
            w_b_next = torch.moveaxis(w_b_next, 1, 0).reshape((num_neurons, -1))

            w_a = torch.cat((w_a, w_a_next), dim=1)
            w_b = torch.cat((w_b, w_b_next), dim=1)

            print(f"w_a shape: {w_a.shape}, w_b shape: {w_b.shape}")

            sim_matrix = w_a @ w_b.T

            if method == "quadratic":
                affinity = build_affinity_matrix_vectorized(w_a, w_b, diag=DiagContent.ONES)
                affinity = torch.tensor(affinity).cuda()

                principal_eigenvector = get_principal_eigenvector(affinity)

                if eigenvec_postprocess == EigenvectorPostprocess.LEORDEANU:
                    perm = extract_matching_leordeanu(principal_eigenvector)
                elif eigenvec_postprocess == EigenvectorPostprocess.LAP:
                    perm = extract_matching_lap(principal_eigenvector)

            elif method == "lap":
                perm = solve_linear_assignment_problem(sim_matrix, return_matrix=True)

            else:
                raise ValueError(f"Unknown method: {method}")

            # plt.imshow(perm, cmap="coolwarm")
            # plt.show()

            old_similarity = compute_weights_similarity(sim_matrix, all_perm_indices[p])

            all_perm_indices[p] = perm_matrix_to_perm_indices(perm)

            new_similarity = compute_weights_similarity(sim_matrix, all_perm_indices[p])

            pylogger.info(f"Iteration {iteration}, Permutation {p}: {new_similarity - old_similarity}")

            progress = progress or new_similarity > old_similarity + 1e-12

        if not progress:
            break

    return all_perm_indices

In [80]:
# dicts for permutations and permuted params, D[a][b] refers to the permutation/params to map b -> a
qap_permutations = {symb: {other_symb: None for other_symb in symbols.difference(symb)} for symb in symbols}

In [81]:
restore_original_weights(models, model_orig_weights)

perm_indices = quadratic_matching(
    permutation_spec,
    fixed_model.model.cpu().state_dict(),
    permutee_model.model.cpu().state_dict(),
    max_iter=100,
    verbose=True,
    method="quadratic",
)

qap_permutations[fixed_symbol][permutee_symbol] = perm_indices

qap_permutations[permutee_symbol][fixed_symbol] = get_inverse_permutations(perm_indices)

Weight matching:   0%|          | 0/100 [00:00<?, ?it/s]/tmp/ipykernel_11089/2081193798.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Wa = torch.tensor(Wa)
/tmp/ipykernel_11089/2081193798.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Wb = torch.tensor(Wb)
Weight matching:   0%|          | 0/100 [00:00<?, ?it/s]

Permuting P_1
Permuting the columns of layer1.weight with P_0, shape: torch.Size([512, 512])
w_a shape: torch.Size([512, 1024]), w_b shape: torch.Size([512, 1024])
torch.Size([512, 512, 512, 512]) torch.Size([512, 512, 512, 512])


RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 274877906944 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
from scripts.evaluate_matched_models import evaluate_pair_of_models

restore_original_weights(models, model_orig_weights)

updated_params = {fixed_symbol: {permutee_symbol: None}}

pylogger.info(f"Permuting model {permutee_symbol} into {fixed_symbol}.")

# perms[a, b] maps b -> a
updated_params[fixed_symbol][permutee_symbol] = apply_permutation_to_statedict(
    permutation_spec, qap_permutations[fixed_symbol][permutee_symbol], models[permutee_symbol].model.state_dict()
)
restore_original_weights(models, model_orig_weights)

lambdas = [0.0, 0.5, 1.0]

qap_results = evaluate_pair_of_models(
    models,
    fixed_symbol,
    permutee_symbol,
    updated_params,
    train_loader,
    test_loader,
    lambdas,
    core_cfg,
)

2025-01-03 08:28:42 INFO     Permuting model b into a.                                                   ]8;id=138632;file:///tmp/ipykernel_11089/2808507962.py\__main__]8;;\:]8;id=420655;file:///tmp/ipykernel_11089/2808507962.py#7\7]8;;\

NameError: name 'qap_permutations' is not defined

## Evaluation

In [ ]:
results = {"git_rebasin": gitrebasin_results, "naive": naive_results, "qap": qap_results}
# plot train and test acc

for i, (method, method_results) in enumerate(results.items()):

    test_acc = method_results["test_acc"]
    train_acc = method_results["train_acc"]

    plt.plot(lambdas, train_acc, label=f"{method}", linestyle="solid", color=palette.get_colors(3)[i])
    plt.plot(lambdas, test_acc, label=f"{method}", linestyle="dashed", color=palette.get_colors(3)[i])

plt.legend()
plt.show()

NameError: name 'func_results' is not defined